In [1]:
import torch
import torch.nn as nn
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from collections import OrderedDict
from facenet_pytorch import MTCNN

In [2]:
facenet = MTCNN(image_size=240, margin=0, min_face_size=20)

In [3]:
model = models.vgg16()
fc = nn.Sequential(OrderedDict(
    [
     ('fc1', nn.Linear(25088,4096)),
     ('relu', nn.ReLU()),
     ('Dropout',nn.Dropout(p=0.5)),
     ('fc2', nn.Linear(4096,4096)),
     ('relu', nn.ReLU()),
     ('Dropout',nn.Dropout(p=0.5)),
     ('fc3', nn.Linear(4096,2)),
     ('output', nn.Softmax(dim=1))
    ]
))
model.classifier = fc

In [4]:
model.load_state_dict(torch.load('vgg16-facemask_v3.pt', map_location='cpu'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [12]:
classes = ['Mask','No Mask']

In [16]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    _, frame = cap.read()
    i_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    region = facenet.detect(i_frame)
    try:
        x1, y1, x2, y2 = region[0][0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        crop = i_frame[y1:y2, x1:x2]
        crop = cv2.resize(crop,(224,224))
        crop_tensor = torch.Tensor(crop).view(1,3,224,224)
        out = model(crop_tensor)
        text = classes[torch.argmax(out)]
        if text == 'Mask':
            cv2.rectangle(frame, (x1,y1), (x2, y2), (0, 255, 0), 3) 
            cv2.putText(frame, text, (x1, y1-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        elif text == 'No Mask':
            cv2.rectangle(frame, (x1,y1), (x2, y2), (0, 0, 255), 3) 
            cv2.putText(frame, text, (x1, y1-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (12,36,255), 2)
        cv2.imshow("Frame",frame)
    except:
        print("Oops!!")
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

Oops!!
Oops!!
Oops!!
Oops!!
